In [1]:
import numpy as np
import pandas as pd

from utils import read_pickles, rmse
from utils_score import WRMSSEEvaluator, calc_score_cv

pd.set_option('display.max_columns', 100)

%matplotlib inline

In [6]:
# load csv files
df = pd.read_csv('../input/sales_train_evaluation.csv')
calendar = pd.read_csv('../input/calendar.csv')
prices = pd.read_csv('../input/sell_prices.csv')
sample_sub = pd.read_csv("../input/sample_submission.csv")

sample_sub["order"] = range(sample_sub.shape[0])

In [2]:
oof = pd.read_csv('../output/oof_lgbm_cv_28days.csv')

oof_foods = pd.read_csv('../output/oof_lgbm_cv_foods.csv')
oof_household = pd.read_csv('../output/oof_lgbm_cv_household.csv')
oof_hobbies = pd.read_csv('../output/oof_lgbm_cv_hobbies.csv')

oof_foods_g = pd.read_csv('../output/oof_lgbm_group_k_fold_foods.csv')
oof_household_g = pd.read_csv('../output/oof_lgbm_group_k_fold_household.csv')
oof_hobbies_g = pd.read_csv('../output/oof_lgbm_group_k_fold_hobbies.csv')

In [3]:
oof = pd.read_csv('../output/oof_lgbm_cv_28days.csv')
scores = calc_score_cv(oof)

 92%|█████████▏| 11/12 [00:05<00:00,  1.31it/s]/media/fujiwara/2THDD/M5-Forecasting-Accuracy/src/utils_score.py:56: RuntimeWarning: Mean of empty slice.
  scale.append(((series[1:] - series[:-1]) ** 2).mean())
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 12/12 [00:10<00:00,  1.16it/s]


In [4]:
end_train=1941
cols_valid1 = [f'd_{c}' for c in range(end_train-28+1,end_train+1)]
cols_valid2 = [f'd_{c}' for c in range(end_train-28*2+1,end_train-28+1)]
cols_valid2 = [f'd_{c}' for c in range(end_train-365+1,end_train-365+28+1)]

In [6]:
round(np.mean(scores),6)

0.618596

In [7]:
df_train1 = df.iloc[:, :-28]
df_valid1 = df.iloc[:, -28:]

evaluator1 = WRMSSEEvaluator(df_train1, df_valid1, calendar, prices)

df_train2 = df.iloc[:, :-28*2]
df_valid2 = df.iloc[:, -28*2:-28]

evaluator2 = WRMSSEEvaluator(df_train2, df_valid2, calendar, prices)

df_train3 = df.iloc[:, :-365]
df_valid3 = df.iloc[:, -365:-365+28]

evaluator3 = WRMSSEEvaluator(df_train3, df_valid3, calendar, prices)

 92%|█████████▏| 11/12 [00:05<00:00,  1.33it/s]/media/fujiwara/2THDD/M5-Forecasting-Accuracy/src/utils_score.py:56: RuntimeWarning: Mean of empty slice.
  scale.append(((series[1:] - series[:-1]) ** 2).mean())
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 12/12 [00:10<00:00,  1.20it/s]


In [5]:
oof = pd.melt(oof,id_vars='id',var_name='d',value_name='demand')

In [6]:
oof['d_numeric']=oof['d'].apply(lambda x: str(x)[2:]).astype(int)
valid1 = oof[(oof['d_numeric']>1941-28)&(oof['d_numeric']<=1941)]
valid2 = oof[(oof['d_numeric']>1941-28*2)&(oof['d_numeric']<=1941-28)]
valid3 = oof[(oof['d_numeric']>1941-365)&(oof['d_numeric']<=1941-365+28)]

In [7]:
valid1 = valid1[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid2 = valid2[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid3 = valid3[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [8]:
cols_d1 = [c for c in valid1.columns if 'd_' in c]
valid1 = valid1.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator1.score(valid1[cols_d1])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5078660464328384
cat_id 0.5128658824289202
state_id 0.5185990485844362
dept_id 0.546743222367672
store_id 0.5865948380721752
item_id 0.8575593943805533
['state_id', 'cat_id'] 0.5440950528493967
['state_id', 'dept_id'] 0.578590245245477
['store_id', 'cat_id'] 0.6185037667846648
['store_id', 'dept_id'] 0.6645376682960282
['item_id', 'state_id'] 0.8528208753122966
['item_id', 'store_id'] 0.8464674726840481
0.6362702927865422


In [9]:
cols_d2 = [c for c in valid2.columns if 'd_' in c]
valid2 = valid2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator2.score(valid2[cols_d2])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.23876481681170472
cat_id 0.2932664800507613
state_id 0.3309030441526736
dept_id 0.3757801475414047
store_id 0.4552217259552134
item_id 0.8431094842694788
['state_id', 'cat_id'] 0.39848028144125025
['state_id', 'dept_id'] 0.47419660613327347
['store_id', 'cat_id'] 0.5323033338349706
['store_id', 'dept_id'] 0.5938330362618495
['item_id', 'state_id'] 0.8511709197879268
['item_id', 'store_id'] 0.8464000135889933
0.519452490819125


In [10]:
cols_d3 = [c for c in valid3.columns if 'd_' in c]
valid3 = valid3.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator3.score(valid3[cols_d3])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5351364652296812
cat_id 0.5585523063098284
state_id 0.6247569109831275
dept_id 0.5808966657090066
store_id 0.6751412563742194
item_id 0.9373676687454482
['state_id', 'cat_id'] 0.6381713151941524
['state_id', 'dept_id'] 0.6700222435604216
['store_id', 'cat_id'] 0.698149278512715
['store_id', 'dept_id'] 0.7348639271569829
['item_id', 'state_id'] 0.8953994576864794
['item_id', 'store_id'] 0.8523123679951605
0.7000641552881018


In [11]:
np.mean([0.6362702927865422,0.519452490819125,0.7000641552881018])

0.6185956462979231

In [15]:
oof2 = oof_foods.append(oof_household)
oof2 = oof2.append(oof_hobbies)

In [29]:
oof2['d_numeric']=oof2['d'].apply(lambda x: str(x)[2:]).astype(int)
valid2_1 = oof2[(oof2['d_numeric']>1941-28)&(oof2['d_numeric']<=1941)]
valid2_2 = oof2[(oof2['d_numeric']>1941-28*2)&(oof2['d_numeric']<=1941-28)]
valid2_3 = oof2[(oof2['d_numeric']>1941-365)&(oof2['d_numeric']<=1941-365+28)]

In [30]:
valid2_1 = valid2_1[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid2_2 = valid2_2[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid2_3 = valid2_3[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [37]:
valid2 = oof2[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [31]:
valid2_1 = valid2_1.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator1.score(valid2_1[cols_d1])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5121444185250336
cat_id 0.5235582464825189
state_id 0.5272711967653047
dept_id 0.5534818341478575
store_id 0.5942724591602403
item_id 0.8689089732407096
['state_id', 'cat_id'] 0.5630643276436076
['state_id', 'dept_id'] 0.5983907118039524
['store_id', 'cat_id'] 0.6378993022226473
['store_id', 'dept_id'] 0.6830731891455724
['item_id', 'state_id'] 0.8597160607222496
['item_id', 'store_id'] 0.8500155098176712
0.6476496858064471


In [38]:
valid2 = valid2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator1.score(valid2[cols_d1])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5121444185250336
cat_id 0.5235582464825189
state_id 0.5272711967653047
dept_id 0.5534818341478575
store_id 0.5942724591602403
item_id 0.8689089732407096
['state_id', 'cat_id'] 0.5630643276436076
['state_id', 'dept_id'] 0.5983907118039524
['store_id', 'cat_id'] 0.6378993022226473
['store_id', 'dept_id'] 0.6830731891455724
['item_id', 'state_id'] 0.8597160607222496
['item_id', 'store_id'] 0.8500155098176712
0.6476496858064471


In [32]:
valid2_2 = valid2_2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator2.score(valid2_2[cols_d2])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.2517976415467098
cat_id 0.33776027901238787
state_id 0.34024402784247093
dept_id 0.41735888781365443
store_id 0.4393869525687345
item_id 0.8436682976169463
['state_id', 'cat_id'] 0.42958707634931004
['state_id', 'dept_id'] 0.507937845501746
['store_id', 'cat_id'] 0.5299036093834514
['store_id', 'dept_id'] 0.5977077457508813
['item_id', 'state_id'] 0.8513368254570093
['item_id', 'store_id'] 0.8465023907433411
0.5327659649655535


In [ ]:
valid2_2 = valid2_2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator2.score(valid2_2[cols_d2])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

In [39]:
groups, scores = evaluator3.score(valid2[cols_d3])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5447619502922475
cat_id 0.5529630486180929
state_id 0.6312995521786061
dept_id 0.5721278672947046
store_id 0.6803157356690829
item_id 0.9330058751286124
['state_id', 'cat_id'] 0.6331011075059556
['state_id', 'dept_id'] 0.6571211974699147
['store_id', 'cat_id'] 0.6912383863647799
['store_id', 'dept_id'] 0.7242997080644101
['item_id', 'state_id'] 0.8923241558560997
['item_id', 'store_id'] 0.8506716727404239
0.6969358547652442


In [2]:
# load out of fold files
oof_28days = pd.read_csv("../output/oof_lgbm_28days.csv")
oof_21days = pd.read_csv("../output/oof_lgbm_21days.csv")
oof_14days = pd.read_csv("../output/oof_lgbm_14days.csv")
oof_7days  = pd.read_csv("../output/oof_lgbm_7days.csv")

# to pivot
oof_28days = oof_28days.pivot(index='id', columns='d', values='demand').reset_index()
oof_21days = oof_21days.pivot(index='id', columns='d', values='demand').reset_index()
oof_14days = oof_14days.pivot(index='id', columns='d', values='demand').reset_index()
oof_7days  = oof_7days.pivot(index='id', columns='d', values='demand').reset_index()

In [19]:
valid_col_28days_fold1 = [f'd_{i+1}' for i in range(1913+21,1913+28)]
valid_col_21days_fold1 = [f'd_{i+1}' for i in range(1913+14,1913+21)]
valid_col_14days_fold1 = [f'd_{i+1}' for i in range(1913+7,1913+14)]
valid_col_7days_fold1  = [f'd_{i+1}' for i in range(1913,1913+7)]

valid_col_28days_fold2 = [f'd_{i+1}' for i in range(1885+21,1885+28)]
valid_col_21days_fold2 = [f'd_{i+1}' for i in range(1885+14,1885+21)]
valid_col_14days_fold2 = [f'd_{i+1}' for i in range(1885+7,1885+14)]
valid_col_7days_fold2  = [f'd_{i+1}' for i in range(1885,1885+7)]

valid_col_28days_fold3 = [f'd_{i+1}' for i in range(1576+21,1576+28)]
valid_col_21days_fold3 = [f'd_{i+1}' for i in range(1576+14,1576+21)]
valid_col_14days_fold3 = [f'd_{i+1}' for i in range(1576+7,1576+14)]
valid_col_7days_fold3  = [f'd_{i+1}' for i in range(1576,1576+7)]


In [20]:
oof = oof_28days[['id']+valid_col_28days_fold1].merge(oof_28days[['id']+valid_col_28days_fold2],on='id',how='left')
oof = oof.merge(oof_28days[['id']+valid_col_28days_fold3],on='id',how='left')

oof = oof.merge(oof_21days[['id']+valid_col_21days_fold1],on='id',how='left')
oof = oof.merge(oof_21days[['id']+valid_col_21days_fold2],on='id',how='left')
oof = oof.merge(oof_21days[['id']+valid_col_21days_fold3],on='id',how='left')

oof = oof.merge(oof_14days[['id']+valid_col_14days_fold1],on='id',how='left')
oof = oof.merge(oof_14days[['id']+valid_col_14days_fold2],on='id',how='left')
oof = oof.merge(oof_14days[['id']+valid_col_14days_fold3],on='id',how='left')

oof = oof.merge(oof_7days[['id']+valid_col_7days_fold1],on='id',how='left')
oof = oof.merge(oof_7days[['id']+valid_col_7days_fold2],on='id',how='left')
oof = oof.merge(oof_7days[['id']+valid_col_7days_fold3],on='id',how='left')

In [21]:
scores = calc_score_cv(oof)
scores

 92%|█████████▏| 11/12 [00:04<00:00,  1.56it/s]/media/fujiwara/2THDD/M5-Forecasting-Accuracy/src/utils_score.py:56: RuntimeWarning: Mean of empty slice.
  scale.append(((series[1:] - series[:-1]) ** 2).mean())
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 12/12 [00:08<00:00,  1.39it/s]


[0.4700711520309957, 0.48544545331938577, 0.5158661214653393]

In [15]:
valid_col_28days_fold3

['d_1597', 'd_1598', 'd_1599', 'd_1600', 'd_1601', 'd_1602', 'd_1603']

In [16]:
# split oof by folds
end_train=1941
cols_valid1 = [f'd_{c}' for c in range(end_train-28+1,end_train+1)]
cols_valid2 = [f'd_{c}' for c in range(end_train-28*2+1,end_train-28+1)]
cols_valid3 = [f'd_{c}' for c in range(end_train-365+1,end_train-365+28+1)]

In [18]:
cols_valid3

['d_1577',
 'd_1578',
 'd_1579',
 'd_1580',
 'd_1581',
 'd_1582',
 'd_1583',
 'd_1584',
 'd_1585',
 'd_1586',
 'd_1587',
 'd_1588',
 'd_1589',
 'd_1590',
 'd_1591',
 'd_1592',
 'd_1593',
 'd_1594',
 'd_1595',
 'd_1596',
 'd_1597',
 'd_1598',
 'd_1599',
 'd_1600',
 'd_1601',
 'd_1602',
 'd_1603',
 'd_1604']